<a href="https://colab.research.google.com/github/shawrk3387/EVA4_Assignments/blob/master/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import statements

In [13]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary
from tqdm import tqdm

#Defining the Deep Net Structure (Layers and Computational Graph)

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        # Input - [128, 1, 28, 28], Output - [128, 32, 28, 28], RF - [3*3]
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        # Input - [128, 32, 28, 28], Output - [128, 64, 28, 28], RF - [5*5]
        self.pool1 = nn.MaxPool2d(2, 2)
        # Input - [128, 64, 28, 28], Output - [128, 64, 14, 14], RF - [10*10]
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        # Input - [128, 64, 14, 14], Output - [128, 128, 14, 14], RF - [12*12]
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        # Input - [128, 128, 14, 14], Output - [128, 256, 14, 14], RF - [14*14]
        self.pool2 = nn.MaxPool2d(2, 2)
        # Input - [128, 256, 14, 14], Output - [128, 256, 7, 7], RF - [28*28]
        self.conv5 = nn.Conv2d(256, 512, 3)
        # Input - [128, 256, 7, 7], Output - [128, 128, 5, 5], RF - [30*30]
        self.conv6 = nn.Conv2d(512, 1024, 3)
        # Input - [128, 128, 5, 5], Output - [128, 64, 3, 3], RF - [32*32]
        self.conv7 = nn.Conv2d(1024, 10, 3)
        # Input - [128, 64, 3, 3], Output - [128, 10, 1, 1], RF - [34*34]

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

#Setting up the model to run on GPU if available otherwise set it to run on CPU

In [15]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu") 
model = Net().to(device)
summary(model, input_size=(1, 28, 28))  # Display the model summary

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

# Initializing the train_loader and test_loader iterators to fetch training and testing data

In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

'''transforms.Normalize: Normalizes an image with mean and standard deviation
0.1307 is the calculated global mean and 0.3081 is the calculated global standard deviation
of MNIST dataset'''
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))
                    ])), batch_size=batch_size, shuffle=True, **kwargs)


# Model training and testing

In [0]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device) # train image tensor and train label tensor
        optimizer.zero_grad() # Clears the gradient after optimizing an input tensor
        output = model(data) # training output
        loss = F.nll_loss(output, target) # training loss
        loss.backward() # back prop initiated to calculate the gradient of the loss w.r.t the weights
        optimizer.step() # optimizer starts updating the weights using the gradients calculated during back prop
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}') # Creates a progress bar to help track the training process

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device) # test image tensor and test label tensor
            output = model(data) # test output
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [18]:
model = Net().to(device) # Loading the model to the processing device
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # defining the optimizer

for epoch in range(1, 2): 
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.03253614902496338 batch_id=468: 100%|██████████| 469/469 [00:18<00:00, 25.08it/s]



Test set: Average loss: 0.0667, Accuracy: 9794/10000 (98%)

